### 데이터 준비

In [4]:
import os, re
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어 list 형태로 읽어오기
file_path = 'shakespeare.txt'
with open(file_path, 'r') as f:
    raw_corpus = f.read().splitlines()
    
print(raw_corpus[:9])

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', '']


> - 원치 않는 문장
    - 화자가 표기된 문장: `'First Citizen:'`, `'All:'`
        - 문장의 끝이 `:`로 끝남 -> 이를 기준으로 문장 제외
    - 공백인 문장: `''`
        - 길이가 0이라면 제외

In [6]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue # 공백이면 건너뛰기
    if sentence[-1] == ':': continue # 화자라면 건너뛰기
    
    if idx > 9: break # 일단 문장 10개만 확인해보기
    
    print(sentence)

Before we proceed any further, hear me speak.
Speak, speak.
You are all resolved rather to die than to famish?


> **토큰화(Tokenize)** : 문장을 일정한 기준으로 쪼개기
> - 가장 간단하게 띄어쓰기 기준으로 쪼개기

> - `Hi,` 와 같이 문장 부호 -> 문장 부호 양쪽에 공백 추가
> - `First`와 `first`를 다른 단어로 인식 -> 모든 문자 소문자로 변환
> - `ten-year-old` 를 한 단어로 인식 -> 특수 문자 모두 제거

In [15]:
# 1. 소문자로 변환. 양쪽 공백 지우기
# 2. 특수 문자([?.!,¿]) 양쪽에 공백 넣기
    # r" \1 ": \1이란 정규식의 첫 번째 그룹 -> 양쪽에 공백 넣기
# 3. 여러 개 공백 -> 한 개로
# 4. a-zA-Z?.!,¿ 가 아닌 모든 문자는 하나의 공백으로 (특수문자 제거)
# 5. 문장 시작에는 <start>, 끝에는 <end>

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence .  <end>


> - 소스 문장(Source Sentence): 모델의 입력이 되는 문장
    - `<start>`를 없애면
> - 타겟 문장(Traget Sentence): 모델의 출력 문장
    - `<end>`를 없애면

In [16]:
# 정제된 문장 모으기
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ':': continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

corpus[:10]

['<start> before we proceed any further , hear me speak .  <end>',
 '<start> speak , speak .  <end>',
 '<start> you are all resolved rather to die than to famish ?  <end>',
 '<start> resolved . resolved .  <end>',
 '<start> first , you know caius marcius is chief enemy to the people .  <end>',
 '<start> we know t , we know t .  <end>',
 '<start> let us kill him , and we ll have corn at our own price .  <end>',
 '<start> is t a verdict ?  <end>',
 '<start> no more talking on t let it be done away , away !  <end>',
 '<start> one word , good citizens .  <end>']

> `tf.keras.preprocessing.text.Tokenizer` 패키지
> - 정제된 데이터를 토큰화, 단어사전 만들고, 데이터 숫자 변환까지 한 번에
> - **벡터화(vectorize)** , **텐서(tensor)**

In [24]:
def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer 만들기
    # 문장 이미 정제했으니 filters 필요 x
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꾸기
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=7000,
        filters=' ',
        oov_token="<unk>")

    # corpus를 이용해 tokenizer 내부 단어장 완성하기
    tokenizer.fit_on_texts(corpus) # 문자를 받아 리스트 형태로 반환
    
    # 준비한 tokenizer로 corpus를 Tensor로 반환하기
    tensor = tokenizer.texts_to_sequences(corpus)

    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2  143   40 ...    0    0    0]
 [   2  110    4 ...    0    0    0]
 [   2   11   50 ...    0    0    0]
 ...
 [   2  149 4553 ...    0    0    0]
 [   2   34   71 ...    0    0    0]
 [   2  945   34 ...    0    0    0]] <keras.preprocessing.text.Tokenizer object at 0x7f8d88901a50>


In [25]:
print(tensor[:3, :10])

[[   2  143   40  933  140  591    4  124   24  110]
 [   2  110    4  110    5    3    0    0    0    0]
 [   2   11   50   43 1201  316    9  201   74    9]]


In [26]:
for idx in tokenizer.index_word:
    print(idx, ':', tokenizer.index_word[idx])
    d
    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : .
6 : the
7 : and
8 : i
9 : to
10 : of


In [27]:
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높음
src_input = tensor[:, :-1]

# <start> 잘라내기
tgt_input = tensor[:, 1:]

print(src_input[0])
print(tgt_input[0])

[  2 143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0
   0   0]
[143  40 933 140 591   4 124  24 110   5   3   0   0   0   0   0   0   0
   0   0]


> 데이터셋 객체 생성
> - `model.fit()`로 numpy array 데이터셋을 생성하지 않고,
> - `tf.data.Dataset`로 텐서 데이터 객체 생성하기

In [33]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

# 0(<pad>) 포함
VOCAB_SIZE = tokenizer.num_words + 1

# 데이터셋 만들기
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

2022-09-06 16:14:41.423323: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<BatchDataset element_spec=(TensorSpec(shape=(256, 20), dtype=tf.int32, name=None), TensorSpec(shape=(256, 20), dtype=tf.int32, name=None))>

> **정리**
> 1. 정규표현식을 이용한 corpus 생성
> 2. `tf.keras.preprocessing.text.Tokenizer`를 이용해 corpus를 텐서로 변환
> 3. `tf.data.Dataset.from_tensor_slices()`를 이용해 corpus 텐서를 `tf.data.Dataset` 객체로 변환